<a href="https://colab.research.google.com/github/GowriShankarG/MTech_AIML/blob/master/ACI_Assignment_Working_Progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inputs

In [34]:
#The grid given in the assignment
cavegrid = [
  ['1010B', '0011N', '0100N', '1011N', '0011N', '0110F'],
  ['1100N', '1110N', '1000N', '0011N', '0110B', '1100N'],
  ['1100N', '1001N', '0100N', '1011F', '0001N', '0100N'],
  ['1000N', '0111F', '1100N', '1010N', '0011N', '0101N'],
  ['1100N', '1010B', '0001N', '0001N', '0010N', '0110B'],
  ['1101N', '1000N', '0011N', '0110N', '1100N', '1100N'],
  ['1011N', '0101N', '1011N', '0001N', '0100N', '1101N']
  ]

StartNode = (0,2)
GoalNode = (6,4)

functions

In [35]:
# CaveGrid Cell details
class Node:
    '''
    class to provide the details of the node in the cave grid
    '''
    def __init__(self, nodeDtls, currNodePos, goalNodePos, gridSize):
        self.nodeWallDtl = nodeDtls[0:4] # first 4 digits for west,east, north,south wall info
        self.nodeObstacle = nodeDtls[4:5]
        self.currNodePos = currNodePos
        self.xpos = currNodePos[0]
        self.ypos = currNodePos[1]
        self.goal_xpos = goalNodePos[0]
        self.goal_ypos = goalNodePos[1]
        self.grid_rows = gridSize[0]
        self.grid_cols = gridSize[1]

    # heuristic function = manhattan distance for the given problem
    def heurfunc_hx(self):
        return abs(self.goal_xpos - self.xpos) + abs(self.goal_ypos - self.ypos)

    def costfunc_gx(self):
      vgx=0
      if self.nodeObstacle == 'N':
        vgx=3 # only node movement cost
      elif self.nodeObstacle == 'B':
        vgx=4 # node movement cost and bush cost (=3+1)
      elif self.nodeObstacle == 'F':
        vgx=8 # node movement cost and Fire cost (=3+5)
      else:
        vgx=9999 # unknown scenario. high cost is assigned
      return vgx

    def overallCostFunc_fx(self):
        return self.costfunc_gx() + self.heurfunc_hx()

    def isWestSideAllowed(self):
        if self.nodeWallDtl[0] == '1': # 1 digit for West
            return False
        else:
            return True

    def isEastSideAllowed(self):
        if self.nodeWallDtl[1] == '1': # 2 digit for East
            return False
        else:
            return True

    def isNorthSideAllowed(self):
        if self.nodeWallDtl[2] == '1': # 3 digit for North
            return False
        else:
            return True

    def isSouthSideAllowed(self):
        if self.nodeWallDtl[3] == '1': # 4 digit for South
            return False
        else:
            return True

    def goalTest(self):
      '''
        checking goal test. if both goal and current node position is same then
        heuristic value (manahatan disntance) is zero.
      '''
      if (abs(self.goal_xpos - self.xpos) + abs(self.goal_ypos - self.ypos)) == 0:
        return True
      else:
        return False

    def getNeigbors(self):
      ''' get the neigbors of the current node
      '''
      westNeigbor = (self.xpos, self.ypos-1)
      eastNeigbor = (self.xpos, self.ypos+1)
      northNeigbor = (self.xpos-1, self.ypos)
      southNeigbor = (self.xpos+1, self.ypos)

      # checking if the neigbors are out of grid size and if any wall exists
      # ignore neibor if it is outside of the grid and wall exits
      #print( "negibor values before  checking grid ==>"+str(westNeigbor)+","+str(eastNeigbor)+","+str(northNeigbor)+","+str(southNeigbor))
      if ((westNeigbor[0] < 0 or westNeigbor[0] >= self.grid_rows or westNeigbor[1] < 0 or westNeigbor[1] >= self.grid_cols) or (self.isWestSideAllowed()==False)):
        westNeigbor = None

      if (eastNeigbor[0] < 0 or eastNeigbor[0] >= self.grid_rows or eastNeigbor[1] < 0 or eastNeigbor[1] >= self.grid_cols or (self.isEastSideAllowed() == False)):
        eastNeigbor = None

      if (northNeigbor[0] < 0 or northNeigbor[0] >= self.grid_rows or northNeigbor[1] < 0 or northNeigbor[1] >= self.grid_cols or (self.isNorthSideAllowed() == False)):
        northNeigbor = None

      if (southNeigbor[0] < 0 or southNeigbor[0] >= self.grid_rows or southNeigbor[1] < 0 or southNeigbor[1] >= self.grid_cols or (self.isSouthSideAllowed() == False)):
        southNeigbor = None

      neibors = [westNeigbor,eastNeigbor,northNeigbor,southNeigbor]
      # remove neibors with values as none
      neibors = [x for x in neibors if x is not None]

      return neibors


    def nodeDebugInfo(self):
        '''
        node info for debugging purpose
        '''
        print("        -----------------------------------------[DEBUG]--------------------------------------------------------------------------")
        print("        ")
        print("        Current Node Position     ==> ("+str(self.xpos)+","+str(self.ypos)+")")
        print("        Node wall Info           ==> "+self.nodeWallDtl)
        print("        Node Obstacle Info       ==> "+self.nodeObstacle)
        print("        Goal Node Position        ==> ("+str(self.goal_xpos)+","+str(self.goal_ypos)+")")
        print("        Is West side allowed     ==> "+str(self.isWestSideAllowed()))
        print("        Is East side allowed     ==> "+str(self.isEastSideAllowed()))
        print("        Is North side allowed    ==> "+str(self.isNorthSideAllowed()))
        print("        Is South side allowed    ==> "+str(self.isSouthSideAllowed()))
        vgx1 = self.costfunc_gx()
        print("        Node movement cost                            ==> "+str(vgx1))
        vhx1 = self.heurfunc_hx()
        print("        Node heuristic value(manhattan distance)      ==> "+str(vhx1))
        vfx1 = self.overallCostFunc_fx()
        print("        Node overall cost(movement+heuristic)         ==> "+str(vfx1))
        print("         -----------------------North->"+str(self.isNorthSideAllowed())+"<------------------------------")
        print("             West->"+str(self.isWestSideAllowed())+"<                       EAST->"+str(self.isEastSideAllowed())+"<  ")
        print("         -----------------------South->"+str(self.isSouthSideAllowed())+"<------------------------------")
        print("        Goal Test                                      ==> "+str(self.goalTest()))
        print("        Valid Neigbors                                 ==> "+str(self.getNeigbors()))
        print("        grid_xsize ==>"+str(self.grid_rows))
        print("        grid_ysize ==>"+str(self.grid_cols))
        print("        ")
        print("        ---------------------------------------[END DEBUG]----------------------------------------------------------------------------")
        return


In [36]:
# Gridsize
gridsize = (len(cavegrid),len(cavegrid[0]))
print("Dubug : cave grid size => "+str(gridsize))


def getFavarableNode(openset):
  ''' get the favarable node and its cost from the openset
  '''
### TODO - update the fn cost if cost is small with different path and can be a favour node

  # apply node class on each item of openset and get least overall cost function
  favarableNode = openset[0].get('item')
  favarableNodeCost = openset[0].get('fncost')
  favarableParent = openset[0].get('parent')
  tmpFavNd = Node(cavegrid[favarableNode[0]][favarableNode[1]],favarableNode,GoalNode,gridsize)

  for item in openset:
    vitem = item.get('item')
    #print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    #print("Dubug : item => "+str(item))
    tmpNd = Node(cavegrid[vitem[0]][vitem[1]],vitem,GoalNode,gridsize)
    #tmpNd.nodeDebugInfo()
    if tmpNd.overallCostFunc_fx() < tmpFavNd.overallCostFunc_fx():
      tmpFavNd = tmpNd # now the low function cost node is new favourable
      favarableNode = vitem
      favarableNodeCost = tmpNd.overallCostFunc_fx()
      favarableParent = item.get('parent')
      #print("Debug tmpNd  "+str(tmpNd.overallCostFunc_fx()))
      #print("Debug tmpFavNd  "+str(tmpFavNd.overallCostFunc_fx()))


  favarableNodeDict = dict(item=favarableNode, fncost=favarableNodeCost, parent=favarableParent)
  print("Dubug : Favarable Node => "+str(favarableNodeDict))

  return favarableNodeDict


# A* Alogrithm

def a_star_alog():

  # A Start algorithm
  openset = [] # open set
  closedset = [] # close set

  # set start position
  # the data structure node diction to keep function cost and node parent details
  # eg structure: {'item': (1, 1), 'fncost': 11, 'parent': (0, 0)}
  startNdDict = dict(item=StartNode,fncost=9999,parent=None)
  openset.append(startNdDict)
  print("Dubug : Openset at Starting position => "+str(openset))
  print("Dubug : Closedset at Starting position => "+str(closedset))

  i=0
  while(len(openset)>0):  # if there is cell in openset
  #for i in range(12): # temp loop
    print("-------------------------------------------------")
    i=i+1
    print("Interation ---->",i)
    print("-------------------------------------------------")
    # get first value of openset for exploration

    favNdDict = getFavarableNode(openset)
    print("Dubug : Favarable Node Object => "+str(favNdDict))
    currNd = Node(cavegrid[favNdDict.get('item')[0]][favNdDict.get('item')[1]],favNdDict.get('item'),GoalNode,gridsize)
    currNd.nodeDebugInfo()

    pathStr=""
    # check if current node is goal node or not
    isGoalNode = currNd.goalTest()
    if isGoalNode:
      print("Goal Node Found")
      pathStr = str(favNdDict.get('item'))
      # print current node
      print("Debug - Goal Node Details : "+str(currNd))
      currNd.nodeDebugInfo()
      # print closedset
      #print("Debug - Closedset Details : "+str(closedset))
      for item in closedset[::-1]:
        pathStr = pathStr+"-->"+str(item.get('item'))

      print("Debug - Path String : "+pathStr)
      return pathStr


    # remove currND from openset and add to closeset
    openset.remove(favNdDict)
    closedset.append(favNdDict)
    #print("Dubug : ** Openset after removing current node => "+str(openset))
    #print("Dubug : #### Closedset after adding current node => "+str(closedset))

    # potential neigbors
    potentialNeigbors = currNd.getNeigbors()
    print("Dubug : Potential Neigbors => "+str(potentialNeigbors))

    tmpset=[]
    # add potential neigbors to openset
    for item in potentialNeigbors:
      tmpNd = Node(cavegrid[item[0]][item[1]],item,GoalNode,gridsize)
      tmpNdDict = dict(item=item,fncost=tmpNd.overallCostFunc_fx(),parent=favNdDict.get('item'))
      tmpset.append(tmpNdDict)

    # check if tmpset item value is equal to closedset item value, if not add to openset
    t1 = [x for x in tmpset if x.get('item') not in [y.get('item') for y in closedset]]
    openset = openset + t1
    print("Dubug : ## Openset after adding potential Neigbors => "+str(openset))

    # todo - check for no potential neigbors
    if len(openset)<=0:
        print("Goal Node Not Found. No poential neigbors. openset is empty")
        return  "No path found"


  return "No path found"

# call A Star
a_star_alog()

Dubug : cave grid size => (7, 6)
Dubug : Openset at Starting position => [{'item': (0, 2), 'fncost': 9999, 'parent': None}]
Dubug : Closedset at Starting position => []
-------------------------------------------------
Interation ----> 1
-------------------------------------------------
Dubug : Favarable Node => {'item': (0, 2), 'fncost': 9999, 'parent': None}
Dubug : Favarable Node Object => {'item': (0, 2), 'fncost': 9999, 'parent': None}
        -----------------------------------------[DEBUG]--------------------------------------------------------------------------
        
        Current Node Position     ==> (0,2)
        Node wall Info           ==> 0100
        Node Obstacle Info       ==> N
        Goal Node Position        ==> (6,4)
        Is West side allowed     ==> True
        Is East side allowed     ==> False
        Is North side allowed    ==> True
        Is South side allowed    ==> True
        Node movement cost                            ==> 3
        Node heur

'(6, 4)-->(5, 4)-->(4, 4)-->(4, 3)-->(4, 2)-->(3, 2)-->(2, 2)-->(1, 3)-->(1, 2)-->(0, 2)'

# Test

In [37]:
#print start and end node details from grid location
print(cavegrid[StartNode[0]][StartNode[1]])
print(cavegrid[GoalNode[0]][GoalNode[1]])

# a fire location (2,3)
print(cavegrid[2][3])

# a bush location (4,5)
print(cavegrid[4][5])

0100N
0100N
1011F
0110B


In [38]:
currNode = (0,2)
goalNode = (6,4)
n1 = Node(cavegrid[currNode[0]][currNode[1]],currNode,goalNode,(len(cavegrid),len(cavegrid[0])))
n1.nodeDebugInfo()


        -----------------------------------------[DEBUG]--------------------------------------------------------------------------
        
        Current Node Position     ==> (0,2)
        Node wall Info           ==> 0100
        Node Obstacle Info       ==> N
        Goal Node Position        ==> (6,4)
        Is West side allowed     ==> True
        Is East side allowed     ==> False
        Is North side allowed    ==> True
        Is South side allowed    ==> True
        Node movement cost                            ==> 3
        Node heuristic value(manhattan distance)      ==> 8
        Node overall cost(movement+heuristic)         ==> 11
         -----------------------North->True<------------------------------
             West->True<                       EAST->False<  
         -----------------------South->True<------------------------------
        Goal Test                                      ==> False
        Valid Neigbors                                 ==> [(0, 

In [39]:
tmpopenset=[{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}, {'item': (1, 2), 'fncost': 10, 'parent': (0, 2)}]
tempopenset1=[{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}]
t1 = getFavarableNode(tmpopenset)
print(t1)

Dubug : Favarable Node => {'item': (1, 2), 'fncost': 10, 'parent': (0, 2)}
{'item': (1, 2), 'fncost': 10, 'parent': (0, 2)}


In [40]:
c1=[{'item': (0, 2), 'fncost': 9999, 'parent': (1, 2)}]
o1=[{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}, {'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}, {'item': (0, 2), 'fncost': 9999, 'parent': (1, 2)}, {'item': (2, 2), 'fncost': 9, 'parent': (1, 2)}]

# getting o1 values where o1 item value is not c1 item value
o1 = [x for x in o1 if x.get('item') != c1[0].get('item')]
print(o1)


[{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}, {'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}, {'item': (2, 2), 'fncost': 9, 'parent': (1, 2)}]


In [41]:
a1=[{'item': (0, 2), 'fncost': 9999, 'parent': None}, {'item': (1, 2), 'fncost': 10, 'parent': (0, 2)}, {'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}, {'item': (2, 2), 'fncost': 9, 'parent': (1, 2)}, {'item': (3, 2), 'fncost': 8, 'parent': (2, 2)}, {'item': (4, 2), 'fncost': 7, 'parent': (3, 2)}, {'item': (4, 3), 'fncost': 6, 'parent': (4, 2)}, {'item': (4, 4), 'fncost': 5, 'parent': (4, 3)}, {'item': (5, 4), 'fncost': 4, 'parent': (4, 4)}]
for i in a1:
  print(i)

{'item': (0, 2), 'fncost': 9999, 'parent': None}
{'item': (1, 2), 'fncost': 10, 'parent': (0, 2)}
{'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}
{'item': (2, 2), 'fncost': 9, 'parent': (1, 2)}
{'item': (3, 2), 'fncost': 8, 'parent': (2, 2)}
{'item': (4, 2), 'fncost': 7, 'parent': (3, 2)}
{'item': (4, 3), 'fncost': 6, 'parent': (4, 2)}
{'item': (4, 4), 'fncost': 5, 'parent': (4, 3)}
{'item': (5, 4), 'fncost': 4, 'parent': (4, 4)}


In [42]:
b1=[{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}, {'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}, {'item': (2, 1), 'fncost': 10, 'parent': (2, 2)}, {'item': (4, 1), 'fncost': 9, 'parent': (4, 2)}, {'item': (4, 4), 'fncost': 5, 'parent': (4, 3)}, {'item': (3, 3), 'fncost': 7, 'parent': (4, 3)}]
for i in b1:
  print(i)

{'item': (0, 1), 'fncost': 12, 'parent': (0, 2)}
{'item': (1, 3), 'fncost': 9, 'parent': (1, 2)}
{'item': (2, 1), 'fncost': 10, 'parent': (2, 2)}
{'item': (4, 1), 'fncost': 9, 'parent': (4, 2)}
{'item': (4, 4), 'fncost': 5, 'parent': (4, 3)}
{'item': (3, 3), 'fncost': 7, 'parent': (4, 3)}


In [43]:
#for i in range(len(cavegrid)):
  #for j in range(len(cavegrid[0])):
    #print(i,j,cavegrid[i][j])

print("done")

done


In [44]:
openset1=[]
tmpdict = dict(item=(1,1),fncost=11,parent=(0,0))
print(tmpdict)
tmpdict1 = dict(item=(2,1),fncost=11,parent=(0,0))
print(tmpdict1)
tmpdict2 = dict(item=(3,1),fncost=11,parent=(0,0))
print(tmpdict2)

openset1.append(tmpdict)
openset1.append(tmpdict1)
openset1.append(tmpdict2)
print(openset1)

print(openset1[0].get('item'))
print(openset1[0].get('fncost'))
print(openset1[0].get('parent'))

{'item': (1, 1), 'fncost': 11, 'parent': (0, 0)}
{'item': (2, 1), 'fncost': 11, 'parent': (0, 0)}
{'item': (3, 1), 'fncost': 11, 'parent': (0, 0)}
[{'item': (1, 1), 'fncost': 11, 'parent': (0, 0)}, {'item': (2, 1), 'fncost': 11, 'parent': (0, 0)}, {'item': (3, 1), 'fncost': 11, 'parent': (0, 0)}]
(1, 1)
11
(0, 0)
